## Bamboo Weekly 12 | Tourism

The numbers come from the International Trade Administration, part of the US Commerce Department. They keep track of many trade-related numbers, and since tourism is a major part of US trade, they keep track of tourists, too. Their “arrivals program” has been tracking the number and origin of tourists every month for more than 20 years, as described here: https://www.trade.gov/i-94-arrivals-program

In [2]:
# Imports
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from typing import Union, Optional
from urllib.request import urlretrieve
from requests import get
from pathlib import Path

pd.set_option("display.max_columns", None)
print("Setup Complete")

Setup Complete


### Fetch Data

In [25]:
# Option 1 - Using requests module
def url_retrieve(url: str, filename: Path) -> Path:
    file = get(url, allow_redirects=True)
    if file.status_code != 200:
        raise ConnectionError(f"Could not download {url}, error code {file.status_code}")
    filename.write_bytes(file.content)
    return filename


url = "https://www.trade.gov/sites/default/files/2022-02/Monthly-Arrivals-2000-Present.xlsx"
filename = Path("data)/Monthly-Arrivals-2000-Present.xlsx")


In [3]:
# Optio 2 - Using urlretrieve module
url = "https://www.trade.gov/sites/default/files/2022-02/Monthly-Arrivals-2000-Present.xlsx"
file, _ = urlretrieve(url, filename="data/Monthly-Arrivals-2000-Present.xlsx")


### Function-load_and_tweak_countries

In [4]:
# Define a function to load the tweak for the countries df
def load_and_tweak_countries(file: str) -> pd.DataFrame:
    # Rename Headers
    rename_cols = {
        "International Visitors--\n   1) Country of Residence\n   2) 1+ nights in the USA\n   3)  Among qualified visa types": "Country",
        "World \nRegion": "World Region",
        "2023-02\nPreliminary": "2023-02",
        "2023-01\nPreliminary": "2023-01",
    }
    # Load Dataframe
    df = pd.read_excel(file, sheet_name="Monthly", usecols="A:C")

    # Tweak Dataframe
    df = (
        df.drop(columns=[1])
        .dropna(axis="index")
        .rename(columns=rename_cols)
        .reset_index(drop=True)
    )

    return df


df_countries = load_and_tweak_countries(file)
df_countries.head()

Monthly
Monthly Y-o-Y % Change


,Country,World Region
0,Afghanistan,Asia
1,Albania,Eastern Europe
2,Algeria,Africa
3,Andorra,Western Europe
4,Angola,Africa


In [ ]:
df_countries.describe()


In [ ]:
df_countries.info()


### Function-covert_to_datetime

In [ ]:
# Define a function to change headers to datetime with specific date format
def convert_to_datetime(df: pd.DataFrame) -> pd.DataFrame:
    df.columns = pd.to_datetime(df.columns).strftime("%Y-%m")
    df.columns = [pd.to_datetime(col) for col in df.columns]
    return df


# tweak_cols(df_travel)

### Function-load_and_tweak_travel

In [ ]:
# Define a function to load the tweak for the travel df
def load_and_tweak_travel(file: str) -> pd.DataFrame:
    # Rename Headers
    rename_cols = {
        "International Visitors--\n   1) Country of Residence\n   2) 1+ nights in the USA\n   3)  Among qualified visa types": "Country",
        "World \nRegion": "World Region",
        "2023-02\nPreliminary": "2023-02",
        "2023-01\nPreliminary": "2023-01",
    }

    # Columns to drop
    cols_to_drop = [
        1,
        "Unnamed: 281",
        "Unnamed: 282",
        "Unnamed: 283",
        "Unnamed: 284",
        "Unnamed: 285",
        "Unnamed: 286",
        "Unnamed: 287",
        "Unnamed: 288",
        "Unnamed: 289",
        "Unnamed: 290",
        "Unnamed: 291",
        "Unnamed: 292",
        "Unnamed: 293",
        "Unnamed: 294",
        "Notes:",
        " ",
    ]
    # Load Dataframe
    df = pd.read_excel(file, sheet_name="Monthly")

    # Tweak Dataframe
    df = (
        df.rename(columns=rename_cols)
        .drop(columns=cols_to_drop)
        .dropna(axis="index", subset=["Country", "World Region"])
        .fillna(value=0)
        .set_index(["Country", "World Region"])
        .replace(
            {"-": 0, " ": 0, "   ": 0, "---": 0}, regex=True
        )  # Fill values with minus sign, one space, three spaces with 0
        .pipe(convert_to_datetime)  # Convert to datetime
        .astype(int)
    )

    return df


df_travel = load_and_tweak_travel(file)
df_travel

In [ ]:
df_travel.info()


In [ ]:
# Manual check if we have replaced correctly the target values
df_travel.query('Country=="Wallis And Futuna Islands"')


### Q3- In the most recent report, which 10 countries had the greatest number of tourists enter the US?

In [ ]:
# Q3- In the most recent report, which 10 countries had the greatest number of tourists enter the US?
# The latest report is February 2023
df_2023 = df_travel[[df_travel.columns[-1]]].nlargest(
    n=10, columns=[df_travel.columns[-1]]
)
df_2023

### Q4- In the first report, which 10 countries had the greatest number of tourists enter the US?

In [ ]:
# Q4- In the first report, which 10 countries had the greatest number of tourists enter the US?
# The first report is the first column,January 2000
df_2000 = df_travel[[df_travel.columns[0]]].nlargest(
    n=10, columns=[df_travel.columns[0]]
)
df_2000

### Q5-Total the number of tourists from each region in the earliest report vs. the latest report. 
### (Yes, you could get this directly from the original Excel spreadsheet, but I want you to calculate this yourself!) Do we see any changes in the last two decades or so?

In [ ]:
# Q5- Total the number of tourists from each region in the earliest report vs. the latest report.
# (Yes, you could get this directly from the original Excel spreadsheet,
# but I want you to calculate this yourself!) Do we see any changes in the last two decades or so?

# Create a copy
df_travel_ = df_travel.reset_index().copy()

# Get the earliest and latest report
latest_col = df_travel_.columns[-1]  # February 2023
earliet_col = df_travel_.columns[-1]  # Janaury 2000

# Group by Region
df_region = (
    df_travel_.groupby(by=["World Region"])[latest_col]
    .sum()
    .to_frame()
    .sort_values(by=[latest_col], ascending=False)
)

df_region


In [ ]:
df_travel_.columns[2]

In [ ]:
# Q5A- Total the number of tourists from each region in the earliest report vs. the latest report.
# (Yes, you could get this directly from the original Excel spreadsheet,
# but I want you to calculate this yourself!) Do we see any changes in the last two decades or so?

# Create a copy
df_travel_ = df_travel.reset_index().copy()

# Get the earliest and latest report
latest_col = df_travel_.columns[-3]  # December 2022
earliest_col = df_travel_.columns[2]  # Janaury 2000

# Group by Region
df_region = (
    df_travel_.groupby(by=["World Region"])[[earliest_col, latest_col]]
    .sum()
    .sort_values(by=[earliest_col, latest_col], ascending=False)
)

df_region


In [ ]:
# Plot this in bar graph
# Style
plt.style.use("fivethirtyeight")
plt.figure(figsize=(5, 10))
# Sort values
df_region = df_region.sort_values(by=[df_region.columns[1]], ascending=True)
# Plot
ax = df_region.plot(kind="barh", figsize=(10, 10))

# Format
ax.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, loc: f"{int(x):,}"))
plt.show()


#### Function-line_plot()

In [ ]:
def line_plot(
    df: pd.DataFrame,
    x: str,
    y: str,
    hue: str,
    color_map: Optional[Union[str, list, dict]] = None,
    year_interval: int=5,
):
    # Style
    plt.style.use("fivethirtyeight")
    fig, ax = plt.subplots(figsize=(20, 5))

    # Plot
    ax = sns.lineplot(df, x=x, y=y, hue=hue, palette=color_map, alpha=0.9, legend=False)
    # Add the text--for each line, find the end, annotate it with a label, and
    # adjust the chart axes so that everything fits on.
    for line, name in zip(ax.lines, df[hue].unique()):
        y = line.get_ydata()[-1]  # NB: to use start value, set [-1] to [0] instead
        x = line.get_xdata()[-1]
        if not np.isfinite(y):
            y = next(reversed(line.get_ydata()[~line.get_ydata().mask]), float("nan"))
        if not np.isfinite(y) or not np.isfinite(x):
            continue
        text = ax.annotate(
            name,
            xy=(x, y),
            xytext=(2, -2),  # YOU CAN ADJUST THE LOC OF LABEL HERE
            color=line.get_color(),
            xycoords=(ax.get_xaxis_transform(), ax.get_yaxis_transform()),
            textcoords="offset points",
            fontweight="regular",
        )
        text_width = (
            text.get_window_extent(fig.canvas.get_renderer())
            .transformed(ax.transData.inverted())
            .width
        )
        if np.isfinite(text_width):
            ax.set_xlim(ax.get_xlim()[0], text.xy[0] + text_width * 1.05)

    # Format x-axis to view dates properly
    ax.xaxis.set_major_locator(mdates.YearLocator(base=year_interval))
    ax.xaxis.set_major_formatter(mdates.DateFormatter("%b-%Y"))

    # Show plot
    plt.show()
    return

In [ ]:
# Plot per Region

# Group by Region
df_region_all = df_travel_.groupby(by=["World Region"]).sum()
df_region_all


# Pivot
def tweak_(df: pd.DataFrame) -> pd.DataFrame:
    df.columns = ["arrival_date", "region", "num_tourist"]
    return df


# Pivot
df_region_pivot = df_region_all.unstack().reset_index().pipe(tweak_)

df_region_pivot_ = df_region_pivot.loc[
    df_region_pivot["region"].isin(["Western Europe", "Asia", "North America"])
]

line_plot(
    df=df_region_pivot_,
    x="arrival_date",
    y="num_tourist",
    hue="region",
    month_interval=24,
)

In [ ]:
# Manual check if we have replaced correctly the target values
df_travel.query('Country=="Wallis And Futuna Islands"')


### Q6- Have any countries had more month-to-month declines in tourism to the US than increases?

In [ ]:
# Define a function to load the tweak for the travel df
def load_and_tweak_travel_(file: str) -> pd.DataFrame:
    # Rename Headers
    rename_cols = {
        "International Visitors--\n   1) Country of Residence\n   2) 1+ nights in the USA\n   3)  Among qualified visa types": "Country",
        "World \nRegion": "World Region",
        "2023-02\nPreliminary": "2023-02",
        "2023-01\nPreliminary": "2023-01",
    }

    # Columns to drop
    cols_to_drop = [
        1,
        "Unnamed: 281",
        "Unnamed: 282",
        "Unnamed: 283",
        "Unnamed: 284",
        "Unnamed: 285",
        "Unnamed: 286",
        "Unnamed: 287",
        "Unnamed: 288",
        "Unnamed: 289",
        "Unnamed: 290",
        "Unnamed: 291",
        "Notes:",
        " ",
    ]
    # Load Dataframe
    df = pd.read_excel(file, sheet_name="Monthly Y-o-Y % Change")

    # Tweak Dataframe
    df = (
        df.rename(columns=rename_cols)
        .drop(columns=cols_to_drop)
        .dropna(axis="index", subset=["Country", "World Region"])
        .fillna(value=0)
        .set_index(["Country", "World Region"])
        .replace(
            {"-": 0, " ": 0, "   ": 0, "---": 0}, regex=True
        )  # Fill values with minus sign, one space, three spaces with 0
        .pipe(convert_to_datetime)  # Convert to datetime
        .astype(float)
        .multiply(100)
    )

    return df



In [ ]:
# Q6A- Have any countries had more month-to-month declines in tourism to the US than increases?
# One way to solve this is to count the negative and positive percentage per country. 
# then get the variance, if its negative then the country had more declines in tourism

# using the Monthly Y-o-Y % Change
df_travel_yoy = load_and_tweak_travel_(file)
df_travel_yoy.head()

In [ ]:
df_travel_yoy[df_travel_yoy<1]

In [ ]:
# Count
df_travel_yoy = df_travel_yoy.assign(
    negative=df_travel_yoy[df_travel_yoy < 0].count(axis="columns"),
    positive=df_travel_yoy[df_travel_yoy > 0].count(axis="columns"),
)
df_travel_yoy = df_travel_yoy.assign(variance=df_travel_yoy["positive"] - df_travel_yoy["negative"])
df_var = df_travel_yoy[["positive","negative", "variance"]]

df_var.head()

In [ ]:
num_negative = df_var["variance"][df_var["variance"] < 0].count()
perc_negative = num_negative / df_travel_yoy.shape[0]
print(f"The percentage of countries that shows decline in tourism to the US from 2002 to 2022 is {perc_negative:.2%}")

In [ ]:
# Which countries shows decline?
df_var[df_var["variance"] < 0][["variance"]]

In [ ]:
# Q6A- What is the trend in the tourism arrival from asia?

df_asia = (
    df_travel_.query('`World Region`== "Asia"')
    .drop(columns=["World Region"])
    .set_index("Country")
)
df_asia

In [ ]:
# Pivot
def tweak(df: pd.DataFrame) -> pd.DataFrame:
    df.columns = ["arrival_date", "country", "num_tourist"]
    return df

# Pivot
df_asia_pivot = df_asia.unstack().reset_index().pipe(tweak)

df_asia_pivot

#### Plot - df_asia_pivot

In [ ]:
# Plot
ax = df_asia_pivot.plot(kind="line", x="arrival_date", y="num_tourist", figsize=(20, 5))

# Format
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, loc: f"{int(x):,}"))

#### Plot - df_asia_pivot hue by country

In [ ]:
# Using seaborn
# Style
plt.figure(figsize=(20, 5))

# Plot

# ax = sns.lineplot(data=df_asia_pivot, x="arrival_date", y="num_tourist", hue="country")

# Plot
line_plot(
    df=df_asia_pivot,
    x="arrival_date",
    y="num_tourist",
    hue="country",
    year_interval=5,
)

#### Plot - df_asia_pivot filtered country

In [ ]:
# Filtered Country
df_asia_pivot_filtered = df_asia_pivot[
    df_asia_pivot["country"].isin(["Japan", "South Korea", "China", "Philippines"])
]
df_asia_pivot_filtered
# Plot
line_plot(
    df=df_asia_pivot_filtered,
    x="arrival_date",
    y="num_tourist",
    hue="country",
    year_interval=4,
)

In [ ]:
# Filtered Country
df_asia_pivot_filtered = df_asia_pivot[
    df_asia_pivot["country"].isin(["Japan", "South Korea", "China", "Philippines"])
]
df_asia_pivot_filtered
# Color Palette
grey_palette = {"Japan": "gray", "South Korea":"gray", "China":"gray", "Philippines":"magenta"}

# Plot
line_plot(
    df=df_asia_pivot_filtered,
    x="arrival_date",
    y="num_tourist",
    hue="country",
    year_interval=4,
    color_map=grey_palette
)

### Calculate the mean of tourists from each country for each decade. (And yes, the current decade will be listed as December 31st, 2030.)

In [ ]:
df_travel_.head(n=2)


### Q7- Calculate the mean of tourists from each country for each decade. (And yes, the current decade will be listed as December 31st, 2030.)


In [ ]:
# Q7- Calculate the mean of tourists from each country for each decade. (And yes, the current decade will be listed as December 31st, 2030.)

# # Group by country
# df_region_all = (df_travel_.groupby(by=["World Region"]).sum())
# df_region_all


# Pivot
def tweak_(df: pd.DataFrame) -> pd.DataFrame:
    df.columns = ["arrival_date", "country", "avg_num_tourist"]
    return df


# Pivot
df_country_pivot = (
    df_travel_.drop(columns=["World Region"])
    .set_index("Country")
    .unstack()
    .reset_index()
    .pipe(tweak_)
)

df_country_pivot.head()
# Groupby country then by arrival_date with 10 year frequency
df_10y = (
    df_country_pivot.groupby(
        by=["country", pd.Grouper(key="arrival_date", freq="10Y")]
    )[["avg_num_tourist"]]
    .mean()
    .reset_index()
)

df_10y


In [ ]:
df_10y.info()


#### Plot df_ph

In [ ]:

# Filter df
df_ph = df_10y.loc[df_10y["country"].isin(["Philippines", "South Korea"])]
# Convert "arrival_date" column to datetime object
# Style
fig, ax = plt.subplots(figsize=(10, 5))

# using Seaborn
ax = sns.barplot(data=df_ph, x="arrival_date", y="avg_num_tourist", hue="country")

# using pandas built in plot
# ax = df_ph.plot(kind="bar", x="arrival_date", y="avg_num_tourist", figsize=(5,5))

# Format x-axis to view dates properly and y-axis to view values with thousands
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, loc: f"{int(x):,}"))
ax.xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m"))


plt.show()